## Solve power flow with MLP load predictions and weather dependent capacity ratings and heat losses

This notebook does the following:

1. Define path to Master file corresponding to TGW year and scenario
2. Solve Power flow
3. Save lines and Xfer data at each timestep (current, voltage, power)

## Initialize

### Import packages

In [2]:
## Import packages
import time
import datetime
import os
import re
import json
import yaml
import joblib
import numpy as np 
import pandas as pd # to create data frames
import pyarrow as pa
import xarray as xr
import random
import matplotlib.pyplot as plt
import shutil #To enable duplicating files
from opendssdirect import dss
from src import physics_ops
from src import file_ops
from src import opendss_ops
from src import TGW_ops
from src import input_ops
from src import df_ops

### Load config file with scenarios and parameters 

In [1]:
config_file_name = 'opendss_config1'; config_path = f"main_folder/config/{config_file_name}.yaml"; config = input_ops.load_config(config_path)

enable_save = 1  # 1 enable | 0 disable

TGW_years_scenarios = config['TGW_years_scenarios']

demand_mode = config['demand_mode']
    
aggregation_level = config['aggregation_level']

CITY_REGIONS_TO_RUN = config['CITY_REGIONS_TO_RUN']

input_data_dict_name = config['input_data_dict_name']
aggregation_level = config['aggregation_level']
smart_ds_year = config['smart_ds_years'][0]
building_types = config["building_types"]

input_data_training_path = config['input_data_training_path']
CITY_REGIONS_TO_RUN = config['CITY_REGIONS_TO_RUN']
start_month = config['start_month']
end_month = config['end_month']

## Initialize parameters for saving paths
Y_column = config['Y_column']
input_data_prediction_path = config['input_data_prediction_path']
output_path_prediction_str = config['output_data_prediction_path']
output_pf_path = config['output_pf_path']

smart_ds_year = config['smart_ds_years'][0]
smart_ds_load_path = config['smart_ds_load_path'] + f"/{smart_ds_year}" # path to procesed smart-ds resstock data 
   
solution_mode = config['solution_mode']

## Define variables to create list of mdh to run
start_month_mdh = config['start_month_mdh'] 
end_month_mdh = config['end_month_mdh']
top_percent_mdh = config['top_percent_mdh']

## Load dictionary, sort by total city aggregated buildings demand, extract mdh of top % load hours
## load demand data 
regional_demand_weather_ampacity_path = smart_ds_load_path + f"/all_cities/aggregated_demand"
regional_demand_weather_ampacity_all_cities = joblib.load(os.path.join(regional_demand_weather_ampacity_path, "regional_demand_weather_ampacity_all_cities"))
## Sort your dictionary by aggregated total demand  
regional_demand_weather_ampacity_all_cities_sorted = df_ops.sort_nested_dict_dfs(regional_demand_weather_ampacity_all_cities, "aggregated_predicted_buildings_total_kw", ascending=False)
top_n_hours = int(np.ceil(8760*top_percent_mdh/100)) # calculate top city demand hours to run (top_percent_mdh% of hours of the year)
## Define start and end load hours to run
start_row_percent = config['start_row_percent']
start_row_idx = int(np.ceil(8760*start_row_percent/100)) # index from which to start loop of load hours
# start_row_idx = 0 # index from which to start loop of load hours
end_row_idx = top_n_hours 

print(f"enable_save:{enable_save} \nsolution_mode: {solution_mode} \nTGW_years_scenarios:{TGW_years_scenarios} \nCITY_REGIONS_TO_RUN: {CITY_REGIONS_TO_RUN}  \n demand mode: {demand_mode}")

print(f"\n\ntop_percent_mdh: {top_percent_mdh}%, top_n_hours: {top_n_hours}, start_row_idx:{start_row_idx} ({start_row_percent}%), end_row_idx:{end_row_idx} ({top_percent_mdh}%) \n")

## Run power flow 
Note: should take about 40min for all scenarios and 3 TGW year-scenario combinations

In [1]:
# Auxilary parameter 
pf_converged = 0 # defult value 0,  0 did not converge | 1 converged

master_file_option = 'TGW' # Options: TGW | default of smart-ds

start_time = time.time()

for TGW_weather_year, TGW_scenarios in TGW_years_scenarios.items():
    for TGW_scenario in TGW_scenarios:
        print(f"--- Starting scenario: {TGW_weather_year} {TGW_scenario} ---\n")
        year = TGW_weather_year; 
        # --- Iterate over all selected regions --- 
        for city, regions in CITY_REGIONS_TO_RUN.items():
            
            ## create list of mdh for top % hours
            df_city = regional_demand_weather_ampacity_all_cities_sorted[(TGW_weather_year, TGW_scenario)][city]   # load regional demand data to later create list of mdh for top % hours  
            list_of_mdh = df_ops.get_top_n_mdh(df_city, top_n_hours, start_month_mdh, end_month_mdh)

            for region in regions:
                print(f"--- Starting scenario: {city} {region} ---\n")
                # --- Iterate over all selected month-day-hour (mdh) --- 
                for row_i in range(start_row_idx, end_row_idx):
                    mdh = list_of_mdh[row_i]; m,d,h = mdh
                    print(f"--- Starting Index: {row_i}, mdh: {mdh} ---\n")

                    # --- Initialize region's paths ---
                    # Create path to regional master file 
                    if solution_mode == 'snapshot':
                        region_path = f"main_folder/SMART-DS/v1.0/{smart_ds_year}/{city}/{region}/scenarios/base_timeseries/opendss_no_loadshapes"
                    else:
                        region_path = f"main_folder/SMART-DS/v1.0/{smart_ds_year}/{city}/{region}/scenarios/base_timeseries/opendss"  

                    # --- Redirect opendss to new master file  ---
                    dss.Basic.ClearAll() 

                    if master_file_option == 'TGW':
                        # --- Redirect opendss to TGW scenario master file  ---
                        new_master_dir = os.path.join(region_path, "predicted_master_files", TGW_scenario, TGW_weather_year)
                        dss.Command(f'Redirect "{new_master_dir}/Master_{TGW_scenario}_{TGW_weather_year}_{m}_{d}_{h}.dss"') # Direct opendss engine to master file 
                    else:
                        # --- Redirect opendss to default master file  ---
                        dss.Command(f'Redirect "{region_path}/Master.dss"') # Direct opendss engine to master file 

                    # --- Solve power flow ---
                    pf_start_time = time.time()
                    dss.Solution.Solve()  # Solve the circuit
                    if dss.Solution.Converged():
                        print(f"---Power flow solution converged successfully after {dss.Solution.Iterations()} iterations---\n")
                        pf_converged = 1
                    else:
                        print("---Power flow solution did not converge!---\n")
                        pf_converged = 0
                    pf_end_time = time.time(); print("PF solve runtime:", (pf_end_time - pf_start_time) / 60, "minutes")

                    # --- Extract line and transformers data (physical properties, loading, voltage etc.) ---
                    dict_bus_coord = opendss_ops.create_dict_bus_coord(region_path)  # Create a dictionary of bus names and coordinates from Buscoords.dss
                    line_df = opendss_ops.extract_line_information(dict_bus_coord, year,m,d,h,row_i) # Extract line data
                    transformer_df = opendss_ops.extract_transformer_information(dict_bus_coord, year,m,d,h,row_i) # Extract transformers data

                    # Add current year line and xfer data to multi-year data frame
                    if row_i == start_row_idx:
                        all_line_df = line_df
                        all_transformer_df = transformer_df
                    else:
                        # concat current df to a multi-year data frame
                        all_line_df = pd.concat([all_line_df, line_df], ignore_index=True)
                        all_transformer_df = pd.concat([all_transformer_df, transformer_df], ignore_index=True) 

                # --- Save results per region---     
                if master_file_option == 'TGW':
                    lines_file_name = f"lines_top_{start_row_percent}_{top_percent_mdh}_percent"
                    transformers_file_name = f"transformers_top_{start_row_percent}_{top_percent_mdh}_percent"
                    metadata_file_name = f"metadata_top_{start_row_percent}_{top_percent_mdh}_percent"
                else:
                    lines_file_name = f"lines_regional_peak_default"
                    transformers_file_name = f"transformers_regional_peak_default"
                    metadata_file_name = f"metadata_regional_peak_default"
                    
                # Initialize dictionaries
                lines_dict = {}
                transformers_dict = {}
                # Initialize inner dictionary per TGW scenario
                lines_dict[(TGW_weather_year,TGW_scenario)] = {}
                transformers_dict[(TGW_weather_year,TGW_scenario)] = {}
                # Add data frames to 2 level key (smart-ds) of dictionaries
                lines_dict[(TGW_weather_year,TGW_scenario)][(smart_ds_year, city, region)] = all_line_df
                transformers_dict[(TGW_weather_year,TGW_scenario)][(smart_ds_year, city, region)] = all_transformer_df
                
                if enable_save == 1:
                    # Save dictionaries as joblib files
                    if master_file_option == 'TGW':
                        predictions_dir = os.path.join(output_pf_path, f"{city}/{region}/{TGW_scenario}/{TGW_weather_year}/")
                    else:
                        predictions_dir = os.path.join(output_pf_path, f"{city}/{region}/smart_ds_default/{smart_ds_year}")

                    os.makedirs(predictions_dir, exist_ok=True) # Create directories if they don't exist
                    
                    if pf_converged == 1: 
                        joblib.dump(lines_dict, os.path.join(predictions_dir, f"{lines_file_name}.joblib"))
                        joblib.dump(transformers_dict, os.path.join(predictions_dir, f"{transformers_file_name}.joblib"))
                    else:
                        joblib.dump(lines_dict, os.path.join(predictions_dir, f"{lines_file_name}_did_not_converge.joblib"))
                        joblib.dump(transformers_dict, os.path.join(predictions_dir, f"{transformers_file_name}_did_not_converge.joblib"))

                    # Define metadata #### !!! Add time array and temp array
                    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
                    metadata = {
                        "changes": "demand_only",
                        "solution_mode": solution_mode,
                        "smart_ds_year": smart_ds_year,
                        "timestamp": timestamp,
                    }
                    # Save metadata to a JSON file
                    metadata_file = f"{predictions_dir}{metadata_file_name}.json"
                    with open(metadata_file, 'w') as f:
                        json.dump(metadata, f)
                    print(f"---- Results for scenario {city} {region} saved succesfuly in {predictions_dir} ----\n")
    
end_time = time.time(); print("Total runtime:", (end_time - start_time) / 60, "minutes")

### Print bus voltages 

In [2]:
buses = dss.Circuit.AllBusNames()
for bus in buses:
    dss.Circuit.SetActiveBus(bus)
    voltages = dss.Bus.puVmagAngle()
    voltages_pu = voltages[::2]
    for v in voltages_pu:
        if v < 0.9 or v > 1.1:
            print(f"Voltage warning at {bus}: {v:.3f} pu")

### Print circuit loads

In [3]:
# Get total losses
losses = dss.Circuit.Losses()
total_loss_kw = losses[0] / 1000  # Convert watts to kW          
# Get total circuit power 
total_circuit_power = dss.Circuit.TotalPower() #  sum of all the powers in the first terminal of each source (Vsource and Isource)
total_circuit_power_kw = - total_circuit_power[0]  # Real power in kW          
# Total load
total_circuit_Loads = total_circuit_power_kw -total_loss_kw

print(f'Total circuit load: {total_circuit_Loads} kw')

### Print line and transformer data

In [2]:
top_10_loading = all_line_df.nlargest(10, 'Loading [%]')
display(top_10_loading)
top_10_loading = all_transformer_df.nlargest(10, 'Loading [%]')
display(top_10_loading)

## Load saved PF data

In [17]:
# --- Save results ---     
lines_file_name = f"lines_regional_peak"
transformers_file_name = f"transformers_regional_peak"
metadata_file_name = f"metadata_regional_peak"

city = 'GSO'
region = 'rural'

TGW_weather_year = '2018'
TGW_scenario = 'historical'
predictions_dir = os.path.join(output_pf_path, f"{city}/{region}/{TGW_scenario}/{TGW_weather_year}/")
lines_dict_1 = joblib.load(os.path.join(predictions_dir, f"{lines_file_name}.joblib"))
transformers_dict_1 = joblib.load(os.path.join(predictions_dir, f"{transformers_file_name}.joblib"))

TGW_weather_year = '2058'
TGW_scenario = 'rcp45hotter'
predictions_dir = os.path.join(output_pf_path, f"{city}/{region}/{TGW_scenario}/{TGW_weather_year}/")
lines_dict_2 = joblib.load(os.path.join(predictions_dir, f"{lines_file_name}.joblib"))
transformers_dict_2 = joblib.load(os.path.join(predictions_dir, f"{transformers_file_name}.joblib"))

lines_dict = lines_dict_1
transformers_dict = transformers_dict_1
lines_dict.update(lines_dict_2)
transformers_dict.update(transformers_dict_2)